# *Summary*

## Data preprocess & analysis
- Download and extract data
- Generalized ETL
- Joining datasets
- External datasets
- Visualizationa and outliers

### Download and extract data & process data
 - First, we read the given datasets, (i.e `transactions` , `consumers` , `merchants`) , we then checked on the data size and data types accordingly, 
 we joined datasets using left outer join, repeated columns had been dropped accordingly
 - We resolve `tag` by saving merchant tags to different columns`field` , `renvenue_leve` and `take_rate`, since `take_rate` is float type
 transform all strings in "field" and "revenue_level" to lowercase
 - Then we store the curated dataframes into both `csv` and `parquet` form (`full_data`)

### External datasets
 - We select SA2(Statistical Area Level 2) data, both `income` and `population` data 
 - After joining these external data, removing null values results in significant loss of data nearly `20%`
  - We use median to fill the null values(missing data), mitigating this issue

### Outlier 
- We check and remove null values 
- Remove unhelpful columns (i.e `address`, `gender`)
- We check for wrong data 
 - i.e `user_id` , `consumer_id` , `postcode` need to be greater than `0`
- Check for features, (i.e `dollar_value` >= 0 , `postcode` == 4)


### Generalized ETL script



 








###  Pyspark initialization and Read data

In [18]:
import argparse
import os
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
import builtins
from cmath import nan


In [19]:
#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

In [20]:
merchants = spark.read.parquet('../data/tables/tbl_merchants.parquet')
consumers = spark.read.parquet('../data/tables/consumer_user_details.parquet')
transactions = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot')
consumers_csv = spark.read.options(header='True', inferSchema='True', delimiter='|').csv('../data/tables/tbl_consumer.csv')

### Joining data set

In [22]:
## left outer join transaction data with consumers data by user_id.
new_transaction = transactions.join(consumers, transactions.user_id == consumers.user_id, "leftouter").drop(consumers.user_id)
new_transaction = new_transaction.join(merchants, new_transaction.merchant_abn == merchants.merchant_abn, "leftouter").drop(merchants.merchant_abn)
new_transaction = new_transaction.join(consumers_csv, new_transaction.consumer_id == consumers_csv.consumer_id, "leftouter").drop(consumers_csv.consumer_id)
new_transaction.limit(5)

user_id,merchant_abn,dollar_value,order_id,order_datetime,consumer_id,name,tags,name,address,state,postcode,gender
5630,60956456424,145.26081329000152,1e14adeb-8e13-44f...,2021-08-21,28242,Ultricies Digniss...,"([gift, card, Nov...",Philip Crawford,7487 Serrano Gard...,NT,841,Undisclosed
5630,48534649627,120.25889985200416,08476339-f383-4ab...,2021-08-15,28242,Dignissim Maecena...,"[[opticians, oPti...",Philip Crawford,7487 Serrano Gard...,NT,841,Undisclosed
5630,60956456424,135.5412540082104,aacfd47a-438b-47f...,2021-08-15,28242,Ultricies Digniss...,"([gift, card, Nov...",Philip Crawford,7487 Serrano Gard...,NT,841,Undisclosed
5630,89932674734,95.37693966478514,6d5790c9-0eef-453...,2021-08-16,28242,Nulla Vulputate C...,((aRtist supply a...,Philip Crawford,7487 Serrano Gard...,NT,841,Undisclosed
5630,14089706307,440.12097711482835,43d1361a-1101-41a...,2021-08-16,28242,Donec Institute,[(computer progra...,Philip Crawford,7487 Serrano Gard...,NT,841,Undisclosed


### Save tags into columns `field`, `revenue_level`, `take_rate`

In [23]:
## save merchant tags to different columns "field", "renvenue_level" and "take_rate", while "take_rate" is float type
## transform all strings in "field" and "revenue_level" to lowercase
new_transaction = new_transaction.withColumn('tags', expr("substring(tags, 3, length(tags)-4)")) \
    .withColumn('field', split(col("tags"), "\], \[|\), \(").getItem(0)) \
        .withColumn('revenue_level', split(col("tags"), "\], \[|\), \(").getItem(1)) \
            .withColumn('take_rate', split(col("tags"), "\], \[|\), \(").getItem(2)) \
                .withColumn('take_rate', regexp_extract(col("take_rate"), r'(\d+).(\d+)', 0)) \
                    .withColumn("take_rate", col('take_rate').cast(FloatType())) \
                        .withColumn('field', lower(col('field'))) \
                            .withColumn('revenue_level', lower(col('revenue_level'))) \
                                .drop("tags")

In [24]:
new_transaction.limit(5)

user_id,merchant_abn,dollar_value,order_id,order_datetime,consumer_id,name,name,address,state,postcode,gender,field,revenue_level,take_rate
5630,60956456424,145.26081329000152,1e14adeb-8e13-44f...,2021-08-21,28242,Ultricies Digniss...,Philip Crawford,7487 Serrano Gard...,NT,841,Undisclosed,"gift, card, novel...",b,4.69
5630,48534649627,120.25889985200416,08476339-f383-4ab...,2021-08-15,28242,Dignissim Maecena...,Philip Crawford,7487 Serrano Gard...,NT,841,Undisclosed,"opticians, optica...",a,6.64
5630,60956456424,135.5412540082104,aacfd47a-438b-47f...,2021-08-15,28242,Ultricies Digniss...,Philip Crawford,7487 Serrano Gard...,NT,841,Undisclosed,"gift, card, novel...",b,4.69
5630,89932674734,95.37693966478514,6d5790c9-0eef-453...,2021-08-16,28242,Nulla Vulputate C...,Philip Crawford,7487 Serrano Gard...,NT,841,Undisclosed,artist supply and...,c,1.67
5630,14089706307,440.12097711482835,43d1361a-1101-41a...,2021-08-16,28242,Donec Institute,Philip Crawford,7487 Serrano Gard...,NT,841,Undisclosed,computer programm...,b,3.33


### Outlier analysis
- drop columns which are not helpful
- Remove null values
- Check for features


In [ ]:
## drop unhelpful columns
cols = ['address','gender', 'consumer_id', 'user_name', 'state']
new_transaction = new_transaction.drop(*cols)

## drop rows that have null values
new_transaction = new_transaction.dropna()

## drop transaction that has dollor value less or equal to 0
new_transaction = new_transaction.filter((col('dollar_value') >= 0))

## check order datetime to in the right range
new_transaction = new_transaction.filter((col('order_datetime') >= '2021-02-28') & (col('order_datetime') <= '2022-08-28'))

## check the consistency of postcode
new_transaction = new_transaction.filter(length(col('postcode')) == 4)

### Combining external dataset (census data)
- We chosed Sa2 population and income as external data, joined external data to main dataset(details in `external.ipynb` & `external2.ipynb`)
- Filter outliers after joining external data
    - fill null values with mean/median, avoid losing too much data
- Distribution plots(null values filled with mean/median)

## Fruad
- Fraud detection 
 - Join fraud data (`consumer_fraud_probability`, `merchant_fraud_probability`) to our main dataset
 - Consider `30%` and above are fraud
    - Add new feature "is_fraud" to classify whether a record is a fraud record
    - Used NaiveBayes model (can be more detailed)
        - high accuracy 


## Ranking system 
- Aim
 - The goal of the model is to recommend to BNPL company the top N cooperative merchants that are in the long-term interest according to some specific characteristics

- Merchant features we are considering 
 - Features of a Merchant
- `transaction_count`: number of transcations made in a specified period.
- `take_rate`: the fee charged by the BNPL firm to a merchant on a transaction. That is, for each transaction made, a certain percentage is taken by the BNPL firm.
- `revenue_level`: `(a, b, c, d, e)` represents the level of revenue bands (unknown to groups) 'a' denotes the smallest band whilst 'e' denotes the highest revenue band.
- `total_revenue`: the total revenue made by a merchant in a specified period
- `mean_consumer_income`: the mean weekly income of each merchant's consumers (used to represents the puchasing power of merchants' target audience)
- `fraud_count`: the number of transactions that are recongnized as fraud
- `main_business_area_popu`: a sum of the number of consumers in the top five postcode areas corresponding to each merchant that has most users within these areas

Further details in rank_model_explain.md 

# Summary of results

- Most important, we perform two ranking systems based on with and without fraud transatcions
- Based on this, we found that fraud will not hugely affect a merchant ranking unless the number of fraud transactions outnumber the normal.
- Before performing the ranking system, we guess the feature 'revenue level' might be the most important. After seeing the real result, we believe it is the most important feature.
- Different segments, have different transaction numbers.
- Merchants sell tents usually has advantages in customer puchasing frequency and overall merchant revenue
- But investing merchants sell jewlery could maximise BNPL earnings per transaction